In [1]:
from pytorchltr.datasets import MSLR10K
train = MSLR10K(split = "train", fold = 2)
test = MSLR10K(split = "test", fold = 2)


In [12]:
import torch

class Model (torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features, 50)
        self.l2 = torch.nn.Linear(50, 10)
        self.l3 = torch.nn.Linear(10, 1)

    def forward(self, x):
        o1 = torch.nn.functional.relu(self.l1(x))
        o2 = torch.nn.functional.relu(self.l2(o1))
        o3 = self.l3(o2)

        return o3

In [13]:
torch.manual_seed(42)
dimensionality = train[0].features.shape[1]
model = Model (dimensionality)

In [10]:
train[0].features.shape

torch.Size([103, 136])

In [14]:
from pytorchltr.loss import PairwiseHingeLoss
optimizer = torch.optim.Adam (model.parameters(), lr = 0.01)
loss_fn = PairwiseHingeLoss ()

In [18]:
from pytorchltr.datasets.list_sampler import UniformSampler

device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam (model.parameters(), lr = 0.01)
for epoch in range(21):
    loader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True, collate_fn = train.collate_fn(UniformSampler()))
    for batch in loader:
        xs, ys, n = batch.features, batch.relevance, batch.n

        xs, ys, n = xs.to(device), ys.to(device), n.to(device)

        loss = loss_fn(model(xs), ys, n).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Finished epoch %d, loss: %f" % (epoch, loss.item()))

Finished epoch 0, loss: 3418.998535
Finished epoch 1, loss: 4069.875000
Finished epoch 2, loss: 4331.878906
Finished epoch 3, loss: 3610.249512
Finished epoch 4, loss: 5661.811035
Finished epoch 5, loss: 4275.103516
Finished epoch 6, loss: 3177.269287
Finished epoch 7, loss: 2717.130615
Finished epoch 8, loss: 4614.614258
Finished epoch 9, loss: 4141.532227
Finished epoch 10, loss: 3779.008057
Finished epoch 11, loss: 5396.676758
Finished epoch 12, loss: 4240.268555
Finished epoch 13, loss: 2320.067383
Finished epoch 14, loss: 3102.223877
Finished epoch 15, loss: 2739.172607
Finished epoch 16, loss: 2900.220947
Finished epoch 17, loss: 2767.623779
Finished epoch 18, loss: 4662.816895
Finished epoch 19, loss: 6637.294434
Finished epoch 20, loss: 5746.135742


In [19]:
from pytorchltr.evaluation import ndcg
loader = torch.utils.data.DataLoader(test, batch_size = 32, shuffle = False, collate_fn = test.collate_fn())

final_score = 0.0

for batch in loader:
    xs, ys, n = batch.features, batch.relevance, batch.n

    xs, ys, n = xs.to(device), ys.to(device), n.to(device)

    scores = model(xs).squeeze()
    final_score += ndcg(scores, ys, n, k = 10).sum().item()

print ("ndcg@10 on test set is: %f" % (final_score/len(test)))

ndcg@10 on test set is: 0.441584
